In [2]:
# !pip install rdflib pandas

import os
import pandas as pd
from rdflib import Graph, RDF, RDFS, OWL, BNode
from rdflib.collection import Collection

from collections import defaultdict, Counter, deque

## OWL Konzepte Vergleich

Zählt für alle 50 BioPortal Ontologien wie oft ein Konzept auftritt und bestimmt unteranderem den Durchschnitt und Median prop Konstrukt

In [ ]:
# Alle Konstrukte
# 🔧 Pfade
ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal\nochmal"
OUT_CSV = "Bio_Portal_ontology_property_counts_full_7.csv"
OUT_SUMMARY = "Bio_Portal_construct_summary_7.csv"

# --- 1) Definition aller zu zählenden Konstrukte ---

# a) Dinge, die über rdf:type gezählt werden (Klassen/Konstrukte + Property-Eigenschaften!)
#    Manche OWL2-Varianten haben alternative Klassennamen (z. B. SymmetricObjectProperty).
TYPE_MULTI = {
    # Klassen-Axiome (TBox)
    "owl:Class": [OWL.Class],
    "rdfs:Class":[RDFS.Class],
    "owl:AllDisjointClasses": [OWL.AllDisjointClasses],
    "owl:DeprecatedClass": [OWL.DeprecatedClass],

    # Individuen-Axiome (ABox)
    "owl:AllDifferent": [OWL.AllDifferent],
    "owl:NegativePropertyAssertion": [OWL.NegativePropertyAssertion],

    # Datatype Handling
    "rdfs:Datatype": [RDFS.Datatype],

    # RDFS Konstrukte (als Typ)
    "rdf:List": [RDF.List],

    # Property-Typen (separat reporten; NICHT in SUMME einbeziehen)
    "owl:ObjectProperty": [OWL.ObjectProperty],
    "owl:DatatypeProperty": [OWL.DatatypeProperty],
    "owl:AnnotationProperty": [OWL.AnnotationProperty],
    "owl:DeprecatedProperty": [OWL.DeprecatedProperty],
    "rdf:Property": [RDF.Property],

    # OWL 2: Axiom-Objekt als Klasse
    "owl:AllDisjointProperties": [OWL.AllDisjointProperties],

    # 👉 Property-Eigenschaften (als Typ!) – hier korrekt via rdf:type:
    "owl:FunctionalProperty": [OWL.FunctionalProperty],
    "owl:InverseFunctionalProperty": [OWL.InverseFunctionalProperty],
    "owl:SymmetricProperty": [OWL.SymmetricProperty],
    "owl:AsymmetricProperty": [OWL.AsymmetricProperty],
    "owl:TransitiveProperty": [OWL.TransitiveProperty],
    "owl:ReflexiveProperty": [OWL.ReflexiveProperty],
    "owl:IrreflexiveProperty": [OWL.IrreflexiveProperty],
}

# b) Dinge, die als Prädikat auftreten (Triple-Mittelteil)
PREDICATES = {
    # Klassen-Axiome (TBox)
    "rdfs:subClassOf": RDFS.subClassOf,
    "owl:equivalentClass": OWL.equivalentClass,
    "owl:disjointWith": OWL.disjointWith,
    "owl:disjointUnionOf": OWL.disjointUnionOf,
    "owl:hasKey": OWL.hasKey,

    # Klassen-Konstruktoren
    "owl:intersectionOf": OWL.intersectionOf,
    "owl:unionOf": OWL.unionOf,
    "owl:complementOf": OWL.complementOf,
    "owl:oneOf": OWL.oneOf,  # EnumeratedClass

    # Property Restrictions
    "owl:someValuesFrom": OWL.someValuesFrom,
    "owl:allValuesFrom": OWL.allValuesFrom,
    "owl:hasValue": OWL.hasValue,
    "owl:minCardinality": OWL.minCardinality,
    "owl:maxCardinality": OWL.maxCardinality,
    "owl:cardinality": OWL.cardinality,
    "owl:minQualifiedCardinality": OWL.minQualifiedCardinality,
    "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality,
    "owl:qualifiedCardinality": OWL.qualifiedCardinality,
    "owl:hasSelf": OWL.hasSelf,

    # Property-Axiome (Prädikate)
    "rdfs:subPropertyOf": RDFS.subPropertyOf,
    "rdfs:domain": RDFS.domain,
    "rdfs:range": RDFS.range,
    "owl:propertyDisjointWith": OWL.propertyDisjointWith,
    "owl:inverseOf": OWL.inverseOf,
    "owl:equivalentProperty": OWL.equivalentProperty,
    "owl:propertyChainAxiom": OWL.propertyChainAxiom,

    # ABox
    "owl:differentFrom": OWL.differentFrom,
    "owl:sameAs": OWL.sameAs,

    # Datatype Handling (Prädikate)
    "owl:onDatatype": OWL.onDatatype,
    "owl:withRestrictions": OWL.withRestrictions,

    # RDFS (Prädikate)
    "rdf:type": RDF.type,
    "rdfs:member": RDFS.member,
    "rdfs:label": RDFS.label,
    "rdfs:comment": RDFS.comment,
    "rdfs:seeAlso": RDFS.seeAlso,
    "rdfs:isDefinedBy": RDFS.isDefinedBy,

    # RDF List Prädikate
    "rdf:first": RDF.first,
    "rdf:rest": RDF.rest,
}
# rdf:nil ist eine Ressource – wir zählen Vorkommen als Objekt separat
COUNT_RDF_NIL = True

# Welche Spalten NICHT in die SUMME eingehen (nur reporten)
EXCLUDE_FROM_SUM = {
    "owl:ObjectProperty",
    "owl:DatatypeProperty",
    "owl:AnnotationProperty",
    "rdf:Property",
    "owl:Class",
    "rdfs:Class"
}

EXTS = (".owl", ".ttl", ".rdf")

def count_in_graph(g: Graph) -> dict:
    counts = {}

    # rdf:type-basierte Zählungen (inkl. Property-Eigenschaften als Typ!)
    for label, class_list in TYPE_MULTI.items():
        total = 0
        for cls in class_list:
            total += sum(1 for _ in g.subjects(RDF.type, cls))
        counts[label] = total

    # Prädikat-basierte Zählungen
    for label, pred in PREDICATES.items():
        counts[label] = sum(1 for _ in g.triples((None, pred, None)))

    # rdf:nil (als Objekt)
    if COUNT_RDF_NIL:
        counts["rdf:nil (as object)"] = sum(1 for _ in g.triples((None, None, RDF.nil)))

    # SUMME (ohne die reinen Property-Typen)
    sum_keys = [k for k in counts.keys() if k not in EXCLUDE_FROM_SUM]
    counts["SUMME"] = sum(counts[k] for k in sum_keys if k != "SUMME")

    return counts

# --- 2) Dateien zählen & CSV mit allen Rohzahlen schreiben ---
rows = []
idx = 1
for fname in sorted(os.listdir(ONT_DIR)):
    print(f"{idx}. {fname}")
    if not fname.lower().endswith(EXTS):
        idx += 1
        continue

    path = os.path.join(ONT_DIR, fname)
    try:
        g = Graph()
        g.parse(path)
        c = count_in_graph(g)
    except Exception as e:
        # bei Fehlern alle Keys auf 0
        zero_keys = (
            list(TYPE_MULTI.keys())
            + list(PREDICATES.keys())
            + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
            + ["SUMME"]
        )
        c = {k: 0 for k in zero_keys}
        c["_error"] = str(e)

    row = {"file": fname, **c}
    print(f"    {row}")
    rows.append(row)
    idx += 1

# Spaltenreihenfolge
columns = (
    ["file"]
    + list(TYPE_MULTI.keys())
    + list(PREDICATES.keys())
    + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
    + ["SUMME"]
)
if any("_error" in r for r in rows):
    columns += ["_error"]

df = pd.DataFrame(rows, columns=columns)
df.to_csv(OUT_CSV, index=False)
print(f"✅ Rohzahlen gespeichert: {OUT_CSV}")

# --- 3) Auswertungstabelle (Summe, #Ontologien, %Ontologien, Ø, Median, Max) ---

# __TOTAL__ Zeilen gibt's hier nicht; wir nehmen direkt df
df = pd.read_csv(OUT_CSV)
N = len(df)

# Alle Konstruktspalten für die Auswertung:
# (Alles außer Datei-Name, _error und SUMME; Property-Typen bleiben drin – werden aber nicht zusammengezählt,
#  hier machen wir eine per-Konstrukt-Auswertung, also dürfen sie erscheinen.)
construct_cols = [c for c in df.columns if c not in ("file", "_error", "SUMME")]

summary_rows = []
for col in construct_cols:
    vals = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
    total = int(vals.sum())
    ont_count = int((vals > 0).sum())
    percent = (ont_count / N * 100) if N else 0

    nonzero_vals = sorted(vals[vals > 0].tolist())
    if ont_count > 0:
        avg = float(vals[vals > 0].mean())
        med = float(vals[vals > 0].median())
        minv = min(nonzero_vals)
        avg_0 = float(vals[vals >= 0].mean())
        med_0 = float(vals[vals >= 0].median())
    else:
        avg = med = minv = avg_0 = med_0 = 0.0
        
    maxv = int(vals.max()) if len(vals) else 0

    summary_rows.append({
        "Construct": col,
        "Summe": total,
        "#Ontologien": ont_count,
        "%Ontologien": round(percent, 1),
        "Ø pro Ontologie (ohne 0)": round(avg, 2),
        "Ø pro Ontologie (mit 0)": round(avg_0, 2),
        "Median (ohne 0)": med,
        "Median (mit 0)": med_0,
        "Min (>0)": minv,
        "Max": maxv,
        "Werte (für Median ohne 0)": nonzero_vals,  # hier alle Werte explizit
        "Werte (für Median mit 0)": sorted(vals.tolist())
    })

summary_df = pd.DataFrame(summary_rows).sort_values(by=["Summe", "#Ontologien"], ascending=False)
summary_df.to_csv(OUT_SUMMARY, index=False)

print(f"✅ Auswertung gespeichert: {OUT_SUMMARY}")
summary_df


Zählt für die 19 BioPortal Ontologien die PropertyChainAxiome haben, aus wie vielen Elementen diese jeweils bestehen

In [ ]:
# PropertyChainAxiom
# 🔧 Pfade
ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal\nochmal"
OUT_CSV = "Bio_Portal_ontology_construct_depth_count3.csv"

# b) Dinge, die als Prädikat auftreten (Triple-Mittelteil)
PREDICATES = {
    "owl:propertyChainAxiom": OWL.propertyChainAxiom,
}

EXTS = (".owl", ".ttl", ".rdf")

def count_in_graph(g: Graph) -> dict:
    counts = {}

    for label, pred in PREDICATES.items():
        results = []
        for super_prop in g.subjects(OWL.propertyChainAxiom, None):
            head = g.value(super_prop, OWL.propertyChainAxiom)
            if head is None:
                continue

            length = 0
            for item in Collection(g, head):
                if isinstance(item, BNode):
                    if isinstance(item, BNode) and g.value(item, OWL.inverseOf) is not None:
                        # inverse Property Expression → zählt als 1
                        length += 1
                    elif isinstance(item, BNode) and g.value(item, OWL.inverseOf) is not None:
                        # ungewöhnlich, aber falls vorhanden: Liste innerhalb der Liste flatten
                        length += sum(1 for _ in Collection(g, item))
                    else:
                        # unbekannter BNode → konservativ als 1 zählen
                        length += 1
                else:
                    # normale URI einer Objekt-Property
                    length += 1

            results.append(length)
        length = dict(Counter(results))
        max_length = max(results) if results else 0
    
        # print(list(Collection(g, triples)))
        counts[label] = sum(1 for _ in g.triples((None, pred, None)))
        counts["maxLength"] = max_length
        counts["length"] = length


    return counts

# --- 2) Dateien zählen & CSV mit allen Rohzahlen schreiben ---
rows = []
idx = 1
for fname in sorted(os.listdir(ONT_DIR)):
    print(f"{idx}. {fname}")
    if not fname.lower().endswith(EXTS):
        idx += 1
        continue

    path = os.path.join(ONT_DIR, fname)
    try:
        g = Graph()
        g.parse(path)
        c = count_in_graph(g)
    except Exception as e:
        # bei Fehlern alle Keys auf 0
        zero_keys = (

            list(PREDICATES.keys())

        )
        c = {k: 0 for k in zero_keys}
        c["_error"] = str(e)

    row = {"file": fname, **c}
    print(f"    {row}")
    rows.append(row)
    idx += 1


# Bekannte Spalten zuerst
known_cols = ["file"] + list(PREDICATES.keys())

# Alle Keys einsammeln, die in rows vorkommen (inkl. max length etc.)
all_keys = set()
for r in rows:
    all_keys.update(r.keys())

# Extras = alles, was nicht schon in known_cols steckt
extra_cols = [k for k in all_keys if k not in known_cols]

# Falls du trotzdem "_error" am Ende haben willst:
if "_error" in extra_cols:
    extra_cols.remove("_error")
    extra_cols = extra_cols + ["_error"]

# Finale Spaltenreihenfolge
columns = known_cols + sorted(extra_cols)

# DataFrame bauen und Spalten erzwingen
df = pd.DataFrame(rows)
df = df.reindex(columns=columns)

df.to_csv(OUT_CSV, index=False)
print(f"✅ Rohzahlen gespeichert: {OUT_CSV}")


Zählt für alle 50 BioPortal Ontologien wie häufig für Proeprties die Konstrukt-Kombinationen Inverse + subProeprty, Inverse + Superproperty und Inverse + Äquivalente Property auftreten

In [30]:
# InverseOf + Sub/SuperPropertyOf und EquivalentProperty
def count_patterns(g):
    sub_inv, super_inv, equiv_inv, func_inv = 0, 0, 0, 0

    # --- Fall 1+2: Direkt modelliert (gleiches Subjekt) ---
    inverses = {x for s, p, o in g.triples((None, OWL.inverseOf, None)) for x in (s, o)}
    for s in inverses:
        # if (s, RDFS.subPropertyOf, None) in g:
        #     sub_inv += 1
        # if (None, RDFS.subPropertyOf, s) in g:
        #     super_inv += 1
        # if (s, OWL.equivalentProperty, None) in g or (None, OWL.equivalentProperty, s) in g:
        #     equiv_inv += 1
        if (s, RDF.type, OWL.FunctionalProperty) in g:
            func_inv +=1

    # # --- Fall 3: BNode-Konstrukte ---
    # for s, _, o in g.triples((None, RDFS.subPropertyOf, None)):
    #     if isinstance(o, BNode) and (o, OWL.inverseOf, None) in g:
    #         sub_inv += 1
    # for s, _, o in g.triples((None, OWL.equivalentProperty, None)):
    #     if isinstance(o, BNode) and (o, OWL.inverseOf, None) in g:
    #         equiv_inv += 1

    return sub_inv, super_inv, equiv_inv, func_inv

# --- Lauf über alle Ontologien ---
ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal"
OUT_CSV = "Bio_Portal_ontology_inverse_combinations_counts_full3.csv"
OUT_SUMMARY = "Bio_Portal_ontology_inverse_combinations_counts_summary3.csv"

results = []
count = 1
for fname in os.listdir(ONT_DIR):
    print(count, fname)
    count += 1
    if not fname.endswith((".ttl", ".rdf", ".owl")):
        continue
    path = os.path.join(ONT_DIR, fname)
    g = Graph()
    try:
        g.parse(path)
        sub_inv, super_inv, equiv_inv, func_inv = count_patterns(g)
    except Exception as e:
        sub_inv, equiv_inv = 0, 0
    results.append({
        "file": fname,
        "subProperty+inverse": sub_inv,
        "superProperty+inverse": super_inv,
        "equivalentProperty+inverse": equiv_inv,
        "functionalProperty+inverse": func_inv
    })

df = pd.DataFrame(results)
df.to_csv(OUT_CSV, index=False)
print(f"✅ Rohzahlen gespeichert: {OUT_CSV}")

# --- Statistik berechnen ---
df = pd.read_csv(OUT_CSV)
N = len(df)

construct_cols = [c for c in df.columns if c not in ("file", "total")]

summary_rows = []
for col in construct_cols:
    vals = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
    total = int(vals.sum())
    ont_count = int((vals > 0).sum())
    percent = (ont_count / N * 100) if N else 0

    nonzero_vals = sorted(vals[vals > 0].tolist())
    if ont_count > 0:
        avg = float(vals[vals > 0].mean())
        med = float(vals[vals > 0].median())
        minv = min(nonzero_vals)
        avg_0 = float(vals[vals >= 0].mean())
        med_0 = float(vals[vals >= 0].median())
    else:
        avg = med = minv = avg_0 = med_0 = 0.0
        
    maxv = int(vals.max()) if len(vals) else 0

    summary_rows.append({
        "Construct": col,
        "Summe": total,
        "#Ontologien": ont_count,
        "%Ontologien": round(percent, 1),
        "Ø pro Ontologie (ohne 0)": round(avg, 2),
        "Ø pro Ontologie (mit 0)": round(avg_0, 2),
        "Median (ohne 0)": med,
        "Median (mit 0)": med_0,
        "Min (> 0)": minv,
        "Max": maxv,
        "Werte (> 0)": nonzero_vals,  # hier alle Werte explizit
        # "Werte (für Median mit 0)": sorted(vals.tolist())
    })

summary_df = pd.DataFrame(summary_rows).sort_values(by=["Summe", "#Ontologien"], ascending=False)
summary_df.to_csv(OUT_SUMMARY, index=False)

print(f"✅ Auswertung gespeichert: {OUT_SUMMARY}")
print("\nZusammenfassung:\n", summary_df)


1 Allergidoc.rdf
2 ATC_with_imports.rdf
3 bao_complete_with_imports.rdf
4 bcio.owl
5 bervo.owl
6 BMONT.owl
7 BTO_with_imports.rdf
8 CDMOnto.owl
9 cheminf_with_imports.rdf
10 cl.owl
11 clo_merged.owl
12 cob.owl
13 COPPER.owl
14 creno_with_imports.rdf
15 CTCAE_5.0.owl
16 dcm_with_imports.rdf
17 DMDSONT.rdf
18 doid_with_imports.rdf
19 EDAM.owl
20 envo.owl
21 GAMUTS.owl
22 gbol-ontology.ttl
23 hspo_with_imports.rdf
24 ICD10CM_with_imports.rdf
25 ICD9CM_with_imports.rdf
26 meo.v.1.0.ttl
27 mwo.owl
28 namo.owl.ttl
29 ncro_with_imports.owl
30 nochmal
31 O3_20250709.owl
32 oae_merged.owl
33 oba.owl
34 obi.owl
35 OBOREL.owl
36 odfa.owl
37 OMIM_with_imports.rdf
38 Ontos mit PropertyChainAxiom
39 ordo_orphanet.owl
40 pato.owl
41 phases.owl
42 plantso.owl
43 RXNORM_with_imports.rdf
44 SCO_with_imports.rdf
45 sio-release.owl
46 sleep.rdf
47 slso.owl
48 SNMI_with_imports.rdf
49 ssbd_v20250512.ttl
50 tara-acupoints-inferred.rdf
51 uberon.owl
52 VANDF_with_imports.rdf
53 without_imports
✅ Rohzahlen ge

Zählt für die 50 BioPortal Ontologien wie oft Kombinationen von Property-Eigenschaften auftreten.\
Es werden nur min einmal gefundene Eigenschaftskombis in die csv aufgenommen. D.h. alle nicht in der csv aufgeführten Kombis kommen in den 50 Ontologien nicht einmal vor. 

In [ ]:
# Property Eigenschaften Kombis
# Ordner mit Ontologien
ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal"
OUT_CSV = "Bio_Portal_ontology_characteristic_combos_counts_full.csv"
OUT_SUMMARY = "Bio_Portal_ontology_characteristic_combos_counts_summary.csv"

EXTS = (".owl", ".ttl", ".rdf", ".jsonld", ".nt", ".n3")

# Property-Eigenschaften
PROP_CHARACTERISTICS = {
    "TransitiveProperty": OWL.TransitiveProperty,
    "FunctionalProperty": OWL.FunctionalProperty,
    "SymmetricProperty": OWL.SymmetricProperty,
    "InverseFunctionalProperty": OWL.InverseFunctionalProperty,
    "IrreflexiveProperty": OWL.IrreflexiveProperty,
    "AsymmetricProperty": OWL.AsymmetricProperty,
    "ReflexiveProperty": OWL.ReflexiveProperty,
}

rows = []
idx = 1
for fname in sorted(os.listdir(ONT_DIR)):
    if not fname.lower().endswith(EXTS):
        continue

    path = os.path.join(ONT_DIR, fname)
    print(f"{idx}. {fname}")
    idx += 1

    try:
        g = Graph()
        g.parse(path)
    except Exception as e:
        print(f"   Fehler beim Parsen: {e}")
        continue

    # Zähler für diese Ontologie
    combo_counts = {}

    for prop in g.subjects(RDF.type, OWL.ObjectProperty):
        characs = []
        for name, owl_type in PROP_CHARACTERISTICS.items():
            if (prop, RDF.type, owl_type) in g:
                characs.append(name)

        if characs:
            combo = "+".join(sorted(characs))
        else:
            combo = "None"  # keine Eigenschaft

        combo_counts[combo] = combo_counts.get(combo, 0) + 1

    # Ergebniszeile für diese Ontologie
    row = {"file": fname}
    row.update(combo_counts)
    rows.append(row)

# DataFrame bauen
df = pd.DataFrame(rows).fillna(0).set_index("file")

# CSV speichern
df.to_csv(OUT_CSV)
print(f"✅ Ergebnisse gespeichert in {OUT_CSV}")

df = pd.read_csv(OUT_CSV)
N = len(df)

# Alle Konstruktspalten für die Auswertung:
# (Alles außer Datei-Name, _error und SUMME; Property-Typen bleiben drin – werden aber nicht zusammengezählt,
#  hier machen wir eine per-Konstrukt-Auswertung, also dürfen sie erscheinen.)
construct_cols = [c for c in df.columns if c not in ("file", "_error", "SUMME")]

summary_rows = []
for col in construct_cols:
    vals = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
    total = int(vals.sum())
    ont_count = int((vals > 0).sum())
    percent = (ont_count / N * 100) if N else 0

    nonzero_vals = sorted(vals[vals > 0].tolist())
    if ont_count > 0:
        avg = float(vals[vals > 0].mean())
        med = float(vals[vals > 0].median())
        minv = min(nonzero_vals)
        avg_0 = float(vals[vals >= 0].mean())
        med_0 = float(vals[vals >= 0].median())
    else:
        avg = med = minv = avg_0 = med_0 = 0.0
        
    maxv = int(vals.max()) if len(vals) else 0

    summary_rows.append({
        "Construct": col,
        "Summe": total,
        "#Ontologien": ont_count,
        "%Ontologien": round(percent, 1),
        "Ø pro Ontologie (ohne 0)": round(avg, 2),
        "Ø pro Ontologie (mit 0)": round(avg_0, 2),
        "Median (ohne 0)": med,
        "Median (mit 0)": med_0,
        "Min (>0)": minv,
        "Max": maxv,
        "Werte (für Median ohne 0)": nonzero_vals,  # hier alle Werte explizit
        "Werte (für Median mit 0)": sorted(vals.tolist())
    })

summary_df = pd.DataFrame(summary_rows).sort_values(by=["Summe", "#Ontologien"], ascending=False)
summary_df.to_csv(OUT_SUMMARY, index=False)

print(f"✅ Auswertung gespeichert: {OUT_SUMMARY}")
summary_df

df.head()


1. ATC_with_imports.rdf
2. Allergidoc.rdf
3. BMONT.owl
4. BTO_with_imports.rdf
5. CDMOnto.owl
6. COPPER.owl
7. CTCAE_5.0.owl
8. DMDSONT.rdf
9. EDAM.owl
10. GAMUTS.owl
11. ICD10CM_with_imports.rdf
12. ICD9CM_with_imports.rdf
13. O3_20250709.owl
14. OBOREL.owl
15. OMIM_with_imports.rdf
16. RXNORM_with_imports.rdf
17. SCO_with_imports.rdf
18. SNMI_with_imports.rdf
19. VANDF_with_imports.rdf
20. bao_complete_with_imports.rdf
21. bcio.owl
22. bervo.owl
23. cheminf_with_imports.rdf
24. cl.owl
25. clo_merged.owl
26. cob.owl
27. creno_with_imports.rdf
28. dcm_with_imports.rdf
29. doid_with_imports.rdf
30. envo.owl
31. gbol-ontology.ttl
32. hspo_with_imports.rdf
33. meo.v.1.0.ttl
34. mwo.owl
35. namo.owl.ttl
36. ncro_with_imports.owl
37. oae_merged.owl
38. oba.owl
39. obi.owl
40. odfa.owl
41. ordo_orphanet.owl
42. pato.owl
43. phases.owl
44. plantso.owl
45. sio-release.owl
46. sleep.rdf
47. slso.owl
48. ssbd_v20250512.ttl
49. tara-acupoints-inferred.rdf
50. uberon.owl
✅ Ergebnisse gespeichert i

,file,None,TransitiveProperty,SymmetricProperty,SymmetricProperty+TransitiveProperty,FunctionalProperty,AsymmetricProperty+IrreflexiveProperty,IrreflexiveProperty+SymmetricProperty,AsymmetricProperty,IrreflexiveProperty,AsymmetricProperty+InverseFunctionalProperty+IrreflexiveProperty,AsymmetricProperty+TransitiveProperty,InverseFunctionalProperty,ReflexiveProperty+TransitiveProperty,ReflexiveProperty+SymmetricProperty,FunctionalProperty+InverseFunctionalProperty
0,ATC_with_imports.rdf,11.0,2.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Allergidoc.rdf,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMONT.owl,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BTO_with_imports.rdf,146.0,0.0,2.0,0.0,0.0,55.0,5.0,2.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0
4,CDMOnto.owl,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
